In [ ]:

import pandas as pd
import statsmodels.api as sm
df = pd.read_csv('FINAL_FINAL.csv')
print(df.head())
print(df.columns)
print(df.shape)




                                     Station Name   11   12  13  14  15   16  \
0                  Secretariat, Amaravati - APPCB  112   92  54  67  85  109   
1                    Gulzarpet, Anantapur - APPCB   44   51  32  61  32   50   
2             Gangineni Cheruvu, Chittoor - APPCB   77  114  55  55  58   55   
3                 Yerramukkapalli, Kadapa - APPCB   78   92  57  61  59   99   
4  Anand Kala Kshetram, Rajamahendravaram - APPCB   95   73  59  77  89  181   

    17   18   19  ...   21   22   23   24   25   26   27   28  \
0  154  122  149  ...  112  107  137  122  109  128  141   83   
1   66   85   92  ...   86  117  117  141   86  122  104  128   
2   96  155  119  ...  111  105  105  150  133  115  144  124   
3  103  109   99  ...  104  108  109  124  108  107  107   62   
4  155  138  126  ...  127  156  144  105  135  169  125  117   

                City           State  
0          Amaravati  Andhra Pradesh  
1          Anantapur  Andhra Pradesh  
2          

In [ ]:
combined_df = pd.read_csv('FINAL_FINAL.csv')
combined_df_long = pd.melt(combined_df, id_vars=['Station Name', 'City', 'State'], 
                           value_vars=['11', '12', '13', '14', '15', '16', '17', '18', '19', '20', 
                                       '21', '22', '23', '24', '25', '26', '27', '28'],
                           var_name='Date', value_name='AQI')

combined_df_long['Date'] = combined_df_long['Date'].astype(int)  
combined_df_long.to_csv('reshaped_aqi_data.csv', index=False)

print(combined_df_long.head())


                                     Station Name               City  \
0                  Secretariat, Amaravati - APPCB          Amaravati   
1                    Gulzarpet, Anantapur - APPCB          Anantapur   
2             Gangineni Cheruvu, Chittoor - APPCB           Chittoor   
3                 Yerramukkapalli, Kadapa - APPCB             Kadapa   
4  Anand Kala Kshetram, Rajamahendravaram - APPCB  Rajamahendravaram   

            State  Date  AQI  
0  Andhra Pradesh    11  112  
1  Andhra Pradesh    11   44  
2  Andhra Pradesh    11   77  
3  Andhra Pradesh    11   78  
4  Andhra Pradesh    11   95  


In [ ]:
reshaped_df = pd.read_csv('reshaped_aqi_data.csv')
reshaped_df['Treated'] = reshaped_df['State'].apply(lambda x: 1 if x == 'Delhi' else 0)
reshaped_df['Date'] = reshaped_df['Date'] - 18
reshaped_df.to_csv('reshaped_aqi_data.csv', index=False)

In [ ]:
combined_df = pd.read_csv('reshaped_aqi_data.csv')

combined_df['Date:Treated'] = combined_df['Date'] * combined_df['Treated']
print(combined_df.isnull().sum())
model = sm.OLS(combined_df['AQI'], sm.add_constant(combined_df[['Date', 'Treated', 'Date:Treated']])).fit()
print(model.summary())
print(model.params['Date:Treated'])



Station Name    0
City            0
State           0
Date            0
AQI             0
Treated         0
Date:Treated    0
dtype: int64
                            OLS Regression Results                            
Dep. Variable:                    AQI   R-squared:                       0.363
Model:                            OLS   Adj. R-squared:                  0.363
Method:                 Least Squares   F-statistic:                     1904.
Date:                Thu, 28 Nov 2024   Prob (F-statistic):               0.00
Time:                        16:26:28   Log-Likelihood:                -57882.
No. Observations:               10008   AIC:                         1.158e+05
Df Residuals:                   10004   BIC:                         1.158e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025   

In [ ]:

df = pd.read_csv('reshaped_aqi_data.csv')
df['Time'] = df['Date'].apply(lambda x: 0 if x <= 0 else 1)
avg_aqi_df = df.groupby(['Station Name', 'City', 'State', 'Time'])['AQI'].mean().reset_index()

pivoted_df = avg_aqi_df.pivot_table(index=['Station Name', 'City', 'State'], columns='Time', values='AQI')
pivoted_df.columns = ['avg_aqi_time_0', 'avg_aqi_time_1']

pivoted_df = pivoted_df.reset_index()
melted_df = pd.melt(pivoted_df, id_vars=['Station Name', 'City', 'State'], 
                    value_vars=['avg_aqi_time_0', 'avg_aqi_time_1'], 
                    var_name='Time', value_name='avg_aqi')
melted_df['Time'] = melted_df['Time'].apply(lambda x: 0 if 'time_0' in x else 1)
melted_df['Treated'] = melted_df['Time'].apply(lambda x: 0 if x == 0 else 1)

melted_df.to_csv('avg_aqi_final.csv', index=False)

print(melted_df[['Station Name', 'City', 'State',  'Time', 'Treated', 'avg_aqi']])

                          Station Name         City           State  Time  \
0           32Bungalows, Bhilai - CECB       Bhilai    Chhattisgarh     0   
1                 AIIMS, Raipur - CECB       Raipur    Chhattisgarh     0   
2         Adarsh Nagar, Jaipur - RSPCB       Jaipur       Rajasthan     0   
3           Airoli, Navi Mumbai - MPCB  Navi Mumbai     Maharashtra     0   
4           Airport Area, Indore - IMC       Indore  Madhya Pradesh     0   
...                                ...          ...             ...   ...   
1107              Worli, Mumbai - MPCB       Mumbai     Maharashtra     1   
1108  Yamunapuram, Bulandshahr - UPPCB  Bulandshahr   Uttar Pradesh     1   
1109   Yerramukkapalli, Kadapa - APPCB       Kadapa  Andhra Pradesh     1   
1110  Zero Point GICI, Gangtok - SSPCB      Gangtok          Sikkim     1   
1111       Zoo Park, Hyderabad - TSPCB    Hyderabad       Telangana     1   

      Treated  avg_aqi  
0           0  103.250  
1           0   97.250  


In [ ]:
df = melted_df
df['Treated'] = df['State'].apply(lambda x: 1 if x == 'Delhi' else 0)
df['Time:Treated'] = df['Time'] * df['Treated']
print(df['Time'].value_counts())
print(df['Treated'].value_counts())
print(df['Time:Treated'].value_counts())

model = sm.OLS(df['avg_aqi'], sm.add_constant(df[['Time', 'Treated', 'Time:Treated']])).fit()

print(model.summary())

Time
0    556
1    556
Name: count, dtype: int64
Treated
0    1034
1      78
Name: count, dtype: int64
Time:Treated
0    1073
1      39
Name: count, dtype: int64
                            OLS Regression Results                            
Dep. Variable:                avg_aqi   R-squared:                       0.436
Model:                            OLS   Adj. R-squared:                  0.434
Method:                 Least Squares   F-statistic:                     285.1
Date:                Thu, 28 Nov 2024   Prob (F-statistic):          3.87e-137
Time:                        17:21:23   Log-Likelihood:                -6274.6
No. Observations:                1112   AIC:                         1.256e+04
Df Residuals:                    1108   BIC:                         1.258e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t   

In [ ]:
combined_df = pd.read_csv('reshaped_aqi_data.csv')
combined_df['D'] = combined_df['State'].apply(lambda x: 1 if x == 'Delhi' else 0)
combined_df = combined_df[combined_df['Date'].isin([0, 7])]

combined_df['T'] = combined_df['Date'].apply(lambda x: 1 if x == 7 else 0)
combined_df['D:T'] = combined_df['D'] * combined_df['T']
model = sm.OLS(combined_df['AQI'], sm.add_constant(combined_df[['D', 'T', 'D:T']])).fit()

print(model.summary())




                            OLS Regression Results                            
Dep. Variable:                    AQI   R-squared:                       0.432
Model:                            OLS   Adj. R-squared:                  0.431
Method:                 Least Squares   F-statistic:                     281.5
Date:                Fri, 29 Nov 2024   Prob (F-statistic):          8.87e-136
Time:                        18:07:16   Log-Likelihood:                -6424.2
No. Observations:                1112   AIC:                         1.286e+04
Df Residuals:                    1108   BIC:                         1.288e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        165.5029      3.442     48.088      0.0

In [ ]:
combined_df = pd.read_csv('reshaped_aqi_data.csv')
combined_df['D'] = combined_df['State'].apply(lambda x: 1 if x == 'Delhi' else 0)
combined_df = combined_df[combined_df['Date'].isin([-7, 0, 7])]
combined_df['P1']= combined_df['Date'].apply(lambda x: 1 if x == 0 else 0)
combined_df['P2']= combined_df['Date'].apply(lambda x: 1 if x == 7 else 0)
combined_df['P1:D'] = combined_df['P1'] * combined_df['D']
combined_df['P2:D'] = combined_df['P2'] * combined_df['D']
model = sm.OLS(combined_df['AQI'], sm.add_constant(combined_df[['D', 'P1', 'P2', 'P1:D', 'P2:D']])).fit()

print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                    AQI   R-squared:                       0.385
Model:                            OLS   Adj. R-squared:                  0.383
Method:                 Least Squares   F-statistic:                     208.3
Date:                Fri, 29 Nov 2024   Prob (F-statistic):          1.04e-172
Time:                        18:02:46   Log-Likelihood:                -9650.9
No. Observations:                1668   AIC:                         1.931e+04
Df Residuals:                    1662   BIC:                         1.935e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        138.2689      3.472     39.822      0.0

In [ ]:

combined_df = pd.read_csv('reshaped_aqi_data.csv')
combined_df['D'] = combined_df['State'].apply(lambda x: 1 if x == 'Delhi' else 0)
combined_df = combined_df[combined_df['Date'].isin([-7 ,-4, 0, 7])]
combined_df['P1'] = combined_df['Date'].apply(lambda x: 1 if x == -4 else 0)
combined_df['P2'] = combined_df['Date'].apply(lambda x: 1 if x == 0 else 0)
combined_df['P3'] = combined_df['Date'].apply(lambda x: 1 if x == 7 else 0)
combined_df['P1:D'] = combined_df['P1'] * combined_df['D']
combined_df['P2:D'] = combined_df['P2'] * combined_df['D']
combined_df['P3:D'] = combined_df['P3'] * combined_df['D']
model = sm.OLS(
    combined_df['AQI'], 
    sm.add_constant(combined_df[['D', 'P1', 'P2', 'P3', 'P1:D', 'P2:D', 'P3:D']])
).fit()
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                    AQI   R-squared:                       0.398
Model:                            OLS   Adj. R-squared:                  0.396
Method:                 Least Squares   F-statistic:                     209.3
Date:                Fri, 29 Nov 2024   Prob (F-statistic):          6.90e-239
Time:                        18:05:42   Log-Likelihood:                -12887.
No. Observations:                2224   AIC:                         2.579e+04
Df Residuals:                    2216   BIC:                         2.584e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        138.2689      3.502     39.486      0.0

In [ ]:
combined_df = pd.read_csv('reshaped_aqi_data.csv')
combined_df['D'] = combined_df['State'].apply(lambda x: 1 if x == 'Bihar' else 0)
combined_df = combined_df[combined_df['Date'].isin([0, 7])]
combined_df['T'] = combined_df['Date'].apply(lambda x: 1 if x == 7 else 0)
combined_df['D:T'] = combined_df['D'] * combined_df['T']
model = sm.OLS(combined_df['AQI'], sm.add_constant(combined_df[['D', 'T', 'D:T']])).fit()

print(model.summary())

c:\Users\halog\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


                            OLS Regression Results                            
Dep. Variable:                    AQI   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                  0.017
Method:                 Least Squares   F-statistic:                     7.408
Date:                Fri, 29 Nov 2024   Prob (F-statistic):           6.48e-05
Time:                        21:55:40   Log-Likelihood:                -6740.2
No. Observations:                1112   AIC:                         1.349e+04
Df Residuals:                    1108   BIC:                         1.351e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        186.2361      4.555     40.883      0.0